# TME1: Indexation 

Indexation = processus informatique consistant à référencer des données (documents,mot-clés,etc..) , de manière à faliciter leur accès ou leur traitement.

In [1]:
import collections
import porter
import math
import re
import TextRepresenter as tr
import json

Exercice1 de compréhension: Indexation d'un petit jeu de donnée

1.1)

In [2]:
doc1="the new home has been saled on top forecasts"
stopwords=["the", "a", "an", "on", "behind", "under", "there", "in", "on"] 
#les stopwords sont les mots vide. Il est inutile de les compter, parceque ce sont des mots qu'on a l'habitude de retrouver
#dans la plupart des documents.
L=doc1.split()


On enlève les stopWords de la liste de mots

In [3]:
def removeStopwords(L,S):
    for e in L:
        if e in S:
            L.remove(e)
    return L

L=removeStopwords(L,stopwords)
#print(L)   

On va normaliser les mots contenus dans la liste.
La normalisation consiste à retirer le suffixe des termes qui se ressemblent, afin de les associer ensemble lors du comptage de mots

In [4]:
def Normaliser(L):
    R=[]
    for e in L:
        R.append(porter.stem(e))
    return R

L=Normaliser(L)
#print(L)

Maintenant pour chaque terme , on doit compter le nombre de fois où il apparait dans le document.
On va utiliser Counter qui va nous faciliter le travail.

Counter:
on peut aussi l'initialiser avec une liste, et chaque élément aura alors un compte de 1 si les éléments ne sont pas répétés

In [5]:
c = collections.Counter(L)
#print(c)

question 1.2)

Le fichier index = il indique pour chaque document,  le nombre d'occurence de chaque termes apparaissant dans le document.
On le représente ici sous la forme d'un dictionnaire. Où chaque id d'un document est représenté par la clé du dictionnaire , et la valeur associée à chaque clé est un dictionnaire , comptant le nombre d'occurence de chaque mot apparaisant dans le document.


In [6]:
doc1="the new home has been saled on top forecasts" 
doc2="the home sales rise in july" 
doc3="there is an increase in home sales in july" 
doc4="july encounter a new home sales rise"
L=[doc1,doc2,doc3,doc4]

index={}
i=0
for d in L:
    E=d.split()
    E=removeStopwords(E,stopwords)
    E=Normaliser(E)
    c = collections.Counter(E)
    index[i]=c
    i+=1

#print(index)
    

Le fichier index inversé = Pour chaque mot rencontré dans les documents , on indique quels sont les documents qui contiennent ce mot, et le nombre d'occurence de ce mot dans le document. On le représente avec un dictionnaire , où les clés sont les mots, et la valeur associé à chaque clé est un dictionnaire indiquant pour chaque document contenant ce mot le nombre d'occurence de ce mot dans le document.

Etape1:on récupère tous les différents mots qui apparaisent dans les documents

In [7]:
M=[]
for d in L:
    E=d.split()
    E=removeStopwords(E,stopwords)
    E=Normaliser(E)
    for e in E:
        if e not in M:
            M.append(e)
#print(M)

In [8]:
indexReverse={}
for m in M:
    d={}
    for c,v in index.items():
        if(m in v):
            d[c]=v[m]
    indexReverse[m]=d
    
#print(indexReverse)

question 1.3)

Term Frequency tf (ti,d) : nombre occurrences de ti dans le document d.
Remarque : varie en fonction de la taille des documents. Si on double la taille des documents, tf double. Le document sera considéré comme plus pertinent.

Inverse Document ferequency idf:

idf (ti)=log( 1+N / 1+ df(ti))

df (ti) : nombre de documents contenant ti 

idf (ti) : fréquence inverse, décroit vers 0 si ti apparait dans tous les documents. ( car log(1)=0 ).

N : nombre de documents

• TF-IDF

xi =tf (ti,d)∗idf (ti)

On va créer un dictionnaire où pour chaque terme ti on calcule le xi associé pour chaque doc où le terme apparait.

In [9]:
D={}
N=4
for m in M:
    d={}
    idf=math.log( (1+N) / (1+len(indexReverse[m])) )
    v=indexReverse[m]
    for c,va in v.items():
        tf=va
        x=tf*idf
        d[c]=x
    D[m]=d
#print(D)

Exercice 2: parsing d'un fichier

Parser un fichier est l’action d’exploiter un fichier sous une forme brute afin d’en tirer les informations utiles.
Afin de faciliter le traitement de l’information, on a tendance à parser le fichier afin d’en soutirer les informations importantes et les insérer dans une base de données.

 buildDocCollectionSimple  lit le ﬁchier ligne par ligne et récupère le contenu des balises.
 On stocke dans un dictionnaire tous les documents contenues dans le fichier brut .txt.
 Pour chaque document on cré un dictionnaire contenant tous les balises qui lui correspond.

In [10]:
def buildDocCollectionSimple(T):
    file = open(T, "r")
    lines = file.readlines()
    file.close()
    D={}
    f=True
    p=True
    i=0
    s=""

    for line in lines:
        if ( re.match(r"^\.I \d+$",line.strip()) ):  
            if(p):
                p=False
            else:
                d[v]=s
                s=""
                D[i]=d
                i+=1

            d={}
            f=True
            m = re.search(r"^\.I (?P<id>\d+)$",line.strip())
            if m is not None:
                v=m.group('id')
                d[".I"]=v
        elif ( re.match(r"^\.\w$",line.strip()) ):
            if(f):
                f=False
                t = re.search(r"^\.(?P<b>\w)$",line.strip())
                if t is not None:
                    v=t.group('b')
                    v="."+v
                    d[v]=""

            else:

                d[v]=s
                s=""
                t = re.search(r"^\.(?P<b>\w)$",line.strip())
                if t is not None:
                    v=t.group('b')
                    v="."+v
                    d[v]=""


        else:
            s=s+line.strip()+" "


    d[v]=s        
    D[i]=d
    return D

In [11]:
D=buildDocCollectionSimple('cacmShort-good.txt')
#print(D)

 Pour chaque document, on affiche son identiﬁant et son texte

In [12]:
#for c,v in D.items():
#    print("identifiant:",v['.I']," texte: ",v['.T'],"\n")

question 2.2)

buildDocumentCollectionRegex récupère le contenu des balises ”.I” et ”.T” de chaque document contenu dans le fichier

In [13]:
def buildDocumentCollectionRegex(T):
    D=buildDocCollectionSimple(T)
    d={}
    for c,v in D.items():
        if('.T' in v):
            d[v['.I']]=v['.T']
        else:
            d[v['.I']]=""
    return d

#print(buildDocumentCollectionRegex('cacmShort-good.txt'))

In [14]:
#print(buildDocumentCollectionRegex('cacm/cacm.txt'))

Exercice 3 – Projet ”Moteur de Recherche” : Etape Indexation

question 3.1)

Les documents sont stockés sous forme d’objets Document pour lesquels on peut accéder aux valeurs de l’identiﬁant et du texte. On peut egalement stocker les autres métadonnées qui permettront d’avoir un aﬃchage plus complet. Implémenter la classe Document qui servira de base au parsing de la collection.


In [15]:
class Document:

    def __init__(self,i,t):
        self.id=i
        self.texte=t

question 3.2)

 la classe Parser qui permet de parser la collection stock´ee sous la forme d’un dictionnaire de Documents.

In [16]:
class Parser:

    def __init__(self):
        self.Dic={}
        
    def parser(self,T):
        D=buildDocumentCollectionRegex2(T)
        for c,v in D.items():
            self.Dic[c]=Document(c,v)
        

question 3.3)

la classe IndexerSimple qui permet d’indexer une collection dans une m´ethode indexation. Pour cela, vous pourrez utiliser la classe TextRepresenter.py associ´ee à la classe porter.py qui permet de normaliser une chaine de caractères.
On souhaite ensuite construire les ﬁchiers index (index + index invers´e) des collections qui ont ´et´e pars´ees. Les ﬁchiers index devront ˆetre ´ecrits dans des ﬁchiers.



buildDocumentCollectionRegex2 récupère le contenu des balises ”.I” et ”.W” de chaque document contenu dans le fichier.
J'en ai fais une deuxieme parceque cette fois le texte est dans la balise .W
Dans cacm.txt , la balise ".W" n'est pas toujours présente pour un document.
Du coups je décide de dire que si il y a pas de texte pour un document , alors je prends la chaine vide .


getTfsForDoc() retourne la représentation (stem-tf) d’un document a partir de l’index. C'est à dire que pour un id de document , on récupère un dictionnaire contenant tous les tf des mots présents dans le document

In [17]:
def buildDocumentCollectionRegex2(T):
    D=buildDocCollectionSimple(T)
    d={}

    for c,v in D.items():
        if('.W' in v):
            d[v['.I']]=v['.W']
        else:
            d[v['.I']]=""
    
    return d

In [18]:
class IndexerSimple:

    def __init__(self):
        self.PS=tr.PorterStemmer() 
        self.index={}
        self.indexReverse={}
    
    def indexation(self,T):
        P=Parser()
        P.parser(T)      
        index={}
        M=[]
        
        for c,v in P.Dic.items():
            s=self.PS.getTextRepresentation(v.texte)
            index[c]=s

            for ca,va in s.items():
                if (ca not in M):
                    M.append(ca) #on stocke tous les différents mots
            
            
        indexReverse={}
        for m in M:
            d={}
            for c,v in index.items():
                if(m in v):
                    d[c]=v[m]
            indexReverse[m]=d
        
        self.index=index
        self.indexReverse=indexReverse
        
        return index,indexReverse
        
    #Avant d'appeller cette méthode il faut appeller la méthode indexation , pour que l'index se cré
    def getTfsForDoc(self,d):
        return self.index[d]
    
    # getTfIDFsForDoc retourne la repr´esentation (stem-TFIDF) d’un document a partir de l’index
    def getTfIDFsForDoc(self,d):
        di={}
        N=len(self.index)
        for c,v in self.index[d].items():
            tf=v
            idf=math.log( (1+N) / (1+len(indexReverse[c])) )
            x=tf*idf
            di[c]=x
        return di
    
    # getTfsForStem qui retourne la repr´esentation (doc-tf) d’un stem a partir de l’index inverse
    def getTfsForStem(self,m):
            return self.indexReverse[m]
    
    # méthode getTfIDFsForStem qui retourne la repr´esentation (doc-TFIDF) d’un stem a partir de l’index inverse
    def getTfIDFsForStem (self,m):
        d={}
        N=len(self.index)
        for c,v in self.indexReverse[m].items():
            r=self.getTfIDFsForDoc(c)
            d[c]=r[m]
        return d
    
    # getStrDoc qui retourne la chaine de caract`ere dont est issu un document donne dans le ﬁchier source
    def getStrDoc(self,d,T):
        D=buildDocumentCollectionRegex2(T)
        return D[d]
        
        

Vu que la taille des fichier est trop grande , pour pouvoir récuperer toute les informations du fichier , il faut lancer jupyter avec la commande : jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

on peut pas afficher tous lindex d'un coup

In [20]:
#I=IndexerSimple()
#index,indexReverse=I.indexation('cisi/cisi.txt')


In [21]:
#print(index['2460'])

Idem pour l'index inversé , on ne peut pas l'afficher d'un coup.

In [22]:
#print(indexReverse['berg'])

Vu qu'on ne peut afficher tous l'index avec un print , on va stocker les informations dans un fichier

In [23]:
# Enregistrer le dictionnaire dans un fichier :
with open('index.txt', 'w') as file:
    json.dump(index, file)

with open('indexInverse.txt', 'w') as file:
    json.dump(indexReverse, file)
 
# Charger le dictionnaire depuis un fichier :
#with open('notes.txt', 'r') as file:
#    notes = json.load(file)

In [24]:
#print(I.getTfsForDoc("201"))